<a href="https://colab.research.google.com/github/yuan7220/ESS/blob/main/Gee_NDVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import leafmap
ee.Authenticate()
ee.Initialize(project='ess-ex1')

In [ ]:
# Replace open the asset ID after uploading the shapefile
Dallas = ee.FeatureCollection('projects/ess-ex1/assets/dallas')
Dallas_poly = Dallas.geometry().bounds()


In [ ]:
# Applies scaling factors.
# Landsat Level 2 data requires scaling factors to convert raw digital numbers to surface reflectance and brightness temperature.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

In [ ]:
# create masks for cloud and cloud shadows
def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    # Bit 3: Cloud Shadow (0 = No, 1 = Yes)
    # Bit 5: Cloud (0 = No, 1 = Yes)
    cloud_shadow_mask = qa.bitwiseAnd(1 << 3).eq(0)
    cloud_mask = qa.bitwiseAnd(1 << 5).eq(0)
    # Combine masks and apply to the image.
    mask = cloud_shadow_mask.And(cloud_mask)
    return image.updateMask(mask)

In [ ]:
col22 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(Dallas_poly).filterDate('2022-01-01', '2022-12-31')
col24 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(Dallas_poly).filterDate('2024-01-01', '2024-12-31')

aoi = Dallas_poly

Dallas_22 = col22.map(lambda image: image.clip(aoi))
Dallas_24 = col24.map(lambda image: image.clip(aoi))

# create cloud free images
masked_Dallas22 = Dallas_22.map(mask_clouds)
masked_Dallas24 = Dallas_24.map(mask_clouds)

# scale data for surface reflectance
Dallas22_scaled = masked_Dallas22.map(apply_scale_factors)
Dallas24_scaled = masked_Dallas24.map(apply_scale_factors)


In [ ]:
# The NormalizedDifference Vegetation Index (NDVI) is calculated using the Near-Infrared (NIR) and Red bands.
# For Landsat 9, these are Band 5 (NIR) and Band 4 (Red). GEE provides a built-in normalizedDifference function.

def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

Dallas22_ndvi = Dallas22_scaled.map(add_ndvi)
Dallas24_ndvi = Dallas24_scaled.map(add_ndvi)


In [ ]:
# Select a single image or create a composite
# Example: Take the median composite of the collection
Dallas22_ndvi_median = Dallas22_ndvi.select('NDVI').median()
Dallas24_ndvi_median = Dallas24_ndvi.select('NDVI').median()

# Define visualization parameters for NDVI
ndvi_palette = ['cyan', 'orange', 'yellow', 'green', 'darkgreen']
ndvi_vis_params = {'min': -0.1, 'max': 0.9, 'palette': ndvi_palette}

# Add the NDVI layer to the map (if using ee.mapclient or similar)
m = geemap.Map()
m.centerObject(Dallas, 10)
m.addLayer(Dallas22_ndvi_median, ndvi_vis_params, '2022 Landsat 9 NDVI')
m.addLayer(Dallas24_ndvi_median, ndvi_vis_params, '2024 Landsat 9 NDVI')
m

Map(center=[32.79352764446268, -96.76653444452349], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
m = geemap.Map(height='800px')
m.centerObject(Dallas, 10)
left_layer = geemap.ee_tile_layer(Dallas22_ndvi_median, ndvi_vis_params, '2022 NDVI')
right_layer = geemap.ee_tile_layer(Dallas24_ndvi_median, ndvi_vis_params, '2024 NDVI')
m.split_map(left_layer, right_layer)
m

Map(center=[32.79352764446268, -96.76653444452349], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
NDVI_diff = Dallas24_ndvi_median.subtract(Dallas22_ndvi_median)

In [ ]:
print(NDVI_diff.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [ ]:
ndvi_vis_params = {'min': -2, 'max': 2, 'palette': ndvi_palette}

In [ ]:
m=geemap.Map(height='800px')
m.centerObject(Dallas, 10)
m.addLayer(NDVI_diff, ndvi_vis_params, 'NDVI difference')
m

Map(center=[32.79352764446268, -96.76653444452349], controls=(WidgetControl(options=['position', 'transparent_…